<a href="https://colab.research.google.com/github/MengOonLee/WebScrapy/blob/master/CountryHoliday/CountryHoliday.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
pip install --no-cache-dir -qU scrapy selenium

In [ ]:
%%writefile CountryHoliday.py
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import wait, expected_conditions
import scrapy
from scrapy import crawler, loader

class CountryHolidayItem(scrapy.Item):
    country = scrapy.Field()
    holiday = scrapy.Field()

class CountryHolidayLoader(loader.ItemLoader):
    item = CountryHolidayItem()

class CountryHolidaySpider(scrapy.Spider):
    name = "CountryHoliday"
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        options = webdriver.chrome.options.Options()
        options.add_argument("--headless")
        options.add_argument("--no-sandbox")
        options.add_argument("--enable-javascript")
        options.add_argument("--disable-cookies")
        options.add_argument("--disable-notifications")
        options.add_argument("--disable-web-security")
        options.add_argument("--incognito")
        self.driver = webdriver.Chrome(options=options)

    def start_requests(self):
        urls = ["https://www.timeanddate.com/holidays/?allcountries"]
        for url in urls:
            request = scrapy.Request(url=url, callback=self.parse_country)
            yield request

    def parse_country(self, response):
        self.driver.get(response.url)

        wait.WebDriverWait(self.driver, timeout=10)\
        .until(expected_conditions.presence_of_element_located(
            (By.XPATH, "//article[@class='category-list']")))

        selector = scrapy.Selector(text=self.driver.page_source)
        for item in selector.css("ul.category-list__list > li"):
            country = item.css("::text").get()
            link = item.css("a::attr(href)").get()
            link += "2024?hol=9"

            loader = CountryHolidayLoader()
            loader.add_value("country", country)

            yield loader.load_item()

            # if link is not None:
            #     yield response.follow(link, callback=self.parse_holiday,
            #         meta={"country":country})

    # def parse_holiday(self, response):
        # self.driver.get(response.url)
        # country = response.meta["country"]
        # print({
        #     "country":country,
        #     "link":response.url
        # })

process = crawler.CrawlerProcess()
    # settings={"FEEDS":{"items.jl":{"format":"jsonlines"}}}
# )
process.crawl(CountryHolidaySpider)
process.start()

In [18]:
%%bash
python CountryHoliday.py

2024-11-14 08:16:50 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
2024-11-14 08:16:50 [scrapy.utils.log] INFO: Versions: lxml 5.3.0.0, libxml2 2.12.9, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.10.0, Python 3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.3, Platform Linux-6.1.85+-x86_64-with-glibc2.35
2024-11-14 08:16:50 [selenium.webdriver.common.selenium_manager] DEBUG: Selenium Manager binary found at: /usr/local/lib/python3.10/dist-packages/selenium/webdriver/common/linux/selenium-manager
2024-11-14 08:16:50 [selenium.webdriver.common.selenium_manager] DEBUG: Executing process: /usr/local/lib/python3.10/dist-packages/selenium/webdriver/common/linux/selenium-manager --browser chrome --language-binding python --output json
2024-11-14 08:16:50 [selenium.webdriver.common.selenium_manager] DEBUG: Driver path: /root/.cache/selenium/chromedriver/linux64/131.0.6778.69/chromedriver
2024-11-14 08

In [ ]:
from selenium import webdriver
from selenium.webdriver.support import wait, expected_conditions
from selenium.webdriver.common.by import By
import scrapy
import time

options = webdriver.chrome.options.Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--enable-javascript")
options.add_argument("--disable-cookies")
options.add_argument("--disable-notifications")
options.add_argument("--disable-web-security")
options.add_argument("--incognito")
driver = webdriver.Chrome(options=options)

url = "https://www.timeanddate.com/holidays/?allcountries"
driver.get(url)

wait.WebDriverWait(driver, timeout=10).until(
    expected_conditions.presence_of_element_located(
    (By.XPATH, "//article[@class='category-list']")))

selector = scrapy.Selector(text=driver.page_source)
# countries = selector.css("ul.category-list__list ::text").getall()
for item in selector.css("ul.category-list__list > li"):
    name = item.css("::text").get()
    link = item.css("a::attr(href)").get() + "2024?hol=9"
    print(name, link)

driver.close()

DEBUG:selenium.webdriver.common.selenium_manager:Selenium Manager binary found at: /usr/local/lib/python3.10/dist-packages/selenium/webdriver/common/linux/selenium-manager
2024-11-14 07:30:07 [selenium.webdriver.common.selenium_manager] DEBUG: Selenium Manager binary found at: /usr/local/lib/python3.10/dist-packages/selenium/webdriver/common/linux/selenium-manager
DEBUG:selenium.webdriver.common.selenium_manager:Executing process: /usr/local/lib/python3.10/dist-packages/selenium/webdriver/common/linux/selenium-manager --browser chrome --language-binding python --output json
2024-11-14 07:30:07 [selenium.webdriver.common.selenium_manager] DEBUG: Executing process: /usr/local/lib/python3.10/dist-packages/selenium/webdriver/common/linux/selenium-manager --browser chrome --language-binding python --output json
DEBUG:selenium.webdriver.common.selenium_manager:Driver path: /root/.cache/selenium/chromedriver/linux64/131.0.6778.69/chromedriver
2024-11-14 07:30:07 [selenium.webdriver.common.sel

Afghanistan /holidays/afghanistan/2024?hol=9
Albania /holidays/albania/2024?hol=9
Algeria /holidays/algeria/2024?hol=9
American Samoa /holidays/american-samoa/2024?hol=9
Andorra /holidays/andorra/2024?hol=9
Angola /holidays/angola/2024?hol=9
Anguilla /holidays/anguilla/2024?hol=9
Antigua and Barbuda /holidays/antigua-and-barbuda/2024?hol=9
Argentina /holidays/argentina/2024?hol=9
Armenia /holidays/armenia/2024?hol=9
Aruba /holidays/aruba/2024?hol=9
Australia /holidays/australia/2024?hol=9
Austria /holidays/austria/2024?hol=9
Azerbaijan /holidays/azerbaijan/2024?hol=9
Bahrain /holidays/bahrain/2024?hol=9
Bangladesh /holidays/bangladesh/2024?hol=9
Barbados /holidays/barbados/2024?hol=9
Belarus /holidays/belarus/2024?hol=9
Belgium /holidays/belgium/2024?hol=9
Belize /holidays/belize/2024?hol=9
Benin /holidays/benin/2024?hol=9
Bermuda /holidays/bermuda/2024?hol=9
Bhutan /holidays/bhutan/2024?hol=9
Bolivia /holidays/bolivia/2024?hol=9
Bosnia and Herzegovina /holidays/bosnia/2024?hol=9
Botsw

DEBUG:urllib3.connectionpool:http://localhost:60987 "DELETE /session/4d12df1860d9d8843cb6510b43912379/window HTTP/11" 200 0
2024-11-14 07:30:16 [urllib3.connectionpool] DEBUG: http://localhost:60987 "DELETE /session/4d12df1860d9d8843cb6510b43912379/window HTTP/11" 200 0
DEBUG:selenium.webdriver.remote.remote_connection:Remote response: status=200 | data={"value":[]} | headers=HTTPHeaderDict({'Content-Length': '12', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
2024-11-14 07:30:16 [selenium.webdriver.remote.remote_connection] DEBUG: Remote response: status=200 | data={"value":[]} | headers=HTTPHeaderDict({'Content-Length': '12', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
DEBUG:selenium.webdriver.remote.remote_connection:Finished Request
2024-11-14 07:30:16 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
